## Using the Planetary Computer's Data API

The Planetary Computer's [`/data` API](https://tomaugspurger.github.io/pc-data-api/lab/) provides an easy way to vizualize and perform basic analytics on data hosted by the Planetary Computer, without having to deploy your own compute in Azure.

### Background

One of the core principals of cloud-native geospatial is putting the compute next to the data. The Planetary Computer stores its data in Azure Blob Storage in the West Europe region, so that would mean using one of Azure's [many compute services](https://docs.microsoft.com/en-us/azure/architecture/guide/technology-choices/compute-decision-tree) to set up your own compute in West Europe. That's  why we set up the [Planetary Computer Hub](https://planetarycomputer.microsoft.com/docs/overview/environment/): a *very* convenient way to get started with cloud-native geospatial from your own browser.

For some use-cases, however, logging into the Hub and starting a Python kernel isn't appropriate (displaying images on a webpage, for example). The Hub is essentially a manual and interactive form of compute, and involves the (costly) process of starting a Jupyter server on a Virtual Machine in Azure. Even if there were a hot virtual machine or Azure Function ready and waiting, eliminating the startup cost, the hassel of deployment might not be worth it for the outcome (displaying an image, again).

That's why the Planetary Computer provides a `/data` API: to efficiently and conveniently serve these kinds of "simple" usecases. The `/data` API, along with our STAC API, is what powers our [Explorer](https://planetarycomputer.microsoft.com/docs/overview/explorer/).

The reference documentation for the data API is at https://planetarycomputer.microsoft.com/api/data/v1/docs. This notebook gives a brief introduction and some examples.

In [1]:
import requests
import pystac
import ipyleaflet
import shapely.geometry
from IPython.display import Image

### Display an Item

The simplest use of the `data` API looks similar to accessing a raw asset from Blob Storage. Many of our STAC items have a `rendered_preview` asset that's actually dynamically served by our `data` API.

In [2]:
r = requests.get(
    "https://planetarycomputer.microsoft.com/api/stac/v1/collections/sentinel-2-l2a/items/S2B_MSIL2A_20220606T080609_R078_T36PUR_20220606T193343"  # noqa: E501
)
item = pystac.Item.from_dict(r.json())
item.assets["rendered_preview"]

<Asset href=https://planetarycomputer.microsoft.com/api/data/v1/item/preview.png?collection=sentinel-2-l2a&item=S2B_MSIL2A_20220606T080609_R078_T36PUR_20220606T193343&assets=visual&asset_bidx=visual%7C1%2C2%2C3&nodata=0>

Notice the `/api/data/v1` in the asset HREF, which indicates that this targets the `/data` API rather than something like `.blob.core.windows.net`, which targets Azure Blob Storage.  A request to that URL will trigger a [TiTiler](https://devseed.com/titiler/) server to read raw data from Blob Storage, combine and transform it (according to the parameters in the URL, which you could customize) and return you the PNG for display.

In [3]:
Image(url=item.assets["rendered_preview"].href)

So we're able to display an asset using a client that only understands HTTP and JSON.

### Display an interactive map 

The `tilejson` asset is similar to `rendered_preview`, but is useful for putting the asset on a map.

In [4]:
item.assets["tilejson"]

<Asset href=https://planetarycomputer.microsoft.com/api/data/v1/item/tilejson.json?collection=sentinel-2-l2a&item=S2B_MSIL2A_20220606T080609_R078_T36PUR_20220606T193343&assets=visual&asset_bidx=visual%7C1%2C2%2C3&nodata=0>

Making a request to that endpoint returns an object with a `tiles` url, which has everything filled in for this specific item.

In [5]:
r = requests.get(item.assets["tilejson"].href).json()
r

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://planetarycomputer.microsoft.com/api/data/v1/item/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?collection=sentinel-2-l2a&item=S2B_MSIL2A_20220606T080609_R078_T36PUR_20220606T193343&assets=visual&asset_bidx=visual%7C1%2C2%2C3&nodata=0'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [31.17569761, 8.95381176, 32.17948101, 9.95039568],
 'center': [31.677589310000002, 9.45210372, 0]}

In [6]:
tiles = r["tiles"][0]

That can be handed to any system that understands tilejson URLs, like `ipyleaflet`. Panning and zooming around the map will trigger requests to load new data.

In [7]:
center = ((item.bbox[1] + item.bbox[3]) / 2, (item.bbox[0] + item.bbox[2]) / 2)

m = ipyleaflet.Map(
    center=center,
    controls=[ipyleaflet.FullScreenControl()],
    zoom=11,
)

m.add_layer(ipyleaflet.TileLayer(url=tiles))
m.scroll_wheel_zoom = True
m

Map(center=[9.45210372, 31.677589310000002], controls=(FullScreenControl(options=['position']), ZoomControl(op…

### Mosaic multiple items

Thus far, we've worked with just a single asset. The `/data` API also supports combining multiple assets into a single asset by [registering a STAC API search](https://planetarycomputer.microsoft.com/api/data/v1/docs#/PgSTAC%20Mosaic%20endpoints/register_search_mosaic_register_post). You define a provide a search defining the space, time, and other properties to include in the results and the `/data` API will combine the results.

We'll define the area of interest as a GeoJSON polygon. The "filter" is defined as [CQL-2 JSON](https://github.com/radiantearth/stac-api-spec/tree/master/fragments/filter#get-query-parameters-and-post-json-fields).

In [8]:
# Define your area of interest
aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [29.036865234375, 7.857940257224196],
            [31.4813232421875, 7.857940257224196],
            [31.4813232421875, 10.055402736564236],
            [29.036865234375, 10.055402736564236],
            [29.036865234375, 7.857940257224196],
        ]
    ],
}

filter_ = {
    "op": "and",
    "args": [
        {"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
        {"op": "=", "args": [{"property": "collection"}, "sentinel-2-l2a"]},
        {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 10]},
    ],
}
collection = "sentinel-2-l2a"

We can register this search with a `POST` request to the `/data/v1/mosaic/register` endpoint.

In [9]:
r_register = requests.post(
    "https://planetarycomputer.microsoft.com/api/data/v1/mosaic/register",
    json={"collection": collection, "filter": filter_},
)
registered = r_register.json()
registered

{'searchid': '125b5f36fbc3a2e80e011495798a9739',
 'links': [{'rel': 'metadata',
   'type': 'application/json',
   'href': 'https://planetarycomputer.microsoft.com/api/data/v1/mosaic/125b5f36fbc3a2e80e011495798a9739/info'},
  {'rel': 'tilejson',
   'type': 'application/json',
   'href': 'https://planetarycomputer.microsoft.com/api/data/v1/mosaic/125b5f36fbc3a2e80e011495798a9739/tilejson.json'}]}

That returns an object with a couple of links. We're interested in the `/tilejson.json` link, to visualize the results on a map.

In [10]:
tilejson_url = registered["links"][1]["href"]

In addition to that `tilejson_url`, we need to provide a couple other things. First, the `collection` ID, which we already have. Second, we need to tell the tiler how to convert the raw data to an image. Several libraries are involved here, including [TiTiler](http://devseed.com/titiler/), [rio-tiler](https://cogeotiff.github.io/rio-tiler/), and [rio-color](https://github.com/mapbox/rio-color). There's a ton of flexibility here, but to to keep things as simple as possible, we'll use the `/data/mosaic/info` to get some good defaults that were set by the Planetary Computer team.

In [11]:
mosaic_info = requests.get(
    "https://planetarycomputer.microsoft.com/api/data/v1/mosaic/info",
    params=dict(collection=item.collection_id),
).json()
render_options = mosaic_info["renderOptions"][0]["options"]
render_options

'assets=B04&assets=B03&assets=B02&nodata=0&color_formula=Gamma RGB 3.2 Saturation 0.8 Sigmoidal RGB 25 0.35'

Finally, we can get our full tilejson URL.

In [12]:
tiles = requests.get(
    f"{tilejson_url}?collection={item.collection_id}&{render_options}"
).json()["tiles"][0]
tiles

'https://planetarycomputer.microsoft.com/api/data/v1/mosaic/tiles/125b5f36fbc3a2e80e011495798a9739/WebMercatorQuad/{z}/{x}/{y}@1x?collection=sentinel-2-l2a&assets=B04&assets=B03&assets=B02&nodata=0&color_formula=Gamma+RGB+3.2+Saturation+0.8+Sigmoidal+RGB+25+0.35'

Which can be provided to `ipyleaflet`.

In [13]:
center = shapely.geometry.shape(aoi).centroid

m = ipyleaflet.Map(
    center=(center.y, center.x),
    controls=[ipyleaflet.FullScreenControl()],
    zoom=9,
)

m.add_layer(ipyleaflet.TileLayer(url=tiles))
m.add_layer(
    ipyleaflet.GeoJSON(data=aoi, style={"fillOpacity": 0}),
)
m.scroll_wheel_zoom = True
m.layout.max_width = "80%"
m

Map(center=[8.956671496894216, 30.25909423828125], controls=(FullScreenControl(options=['position']), ZoomCont…

This is essentially how the [Planetary Computer Explorer](https://planetarycomputer.microsoft.com/explore) works. The `filter` is generated based on your browser's window and whatever filters you've toggled. Based on that user input, it generates the CQL2-json query, registers a search, builds a TileJSON request (using any visualization options you've set) and displays the result on the map.

### Next steps

This was a brief introduction to the `/data` API. For more, see the [reference documentation](https://planetarycomputer.microsoft.com/api/data/v1/docs). Feel free to share your creations using the `/data` API on the Planetary Computer [discussions board](https://github.com/microsoft/PlanetaryComputer/discussions/categories/show-and-tell).